In [1]:
import sys
sys.path.append('..')

from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from enum import Enum
from copy import deepcopy

from scipy.stats import kruskal

from src.services import facebook_creative_descriptive_saver as fcd
from src.database.session import SessionLocal
from src.pingers import *
from src.utils import *
from src import crud
from src.utils import *
from src.abc.descriptive import DescriptiveDF
from src.utils.metric import *
from src.crud.utils.get_performance import get_performance, get_conversion_rates_dict

db = SessionLocal()
end_date_plus_one = date(year=2023, month=8, day=1)
end_date = end_date_plus_one - timedelta(days=1)
start_date = end_date_plus_one - relativedelta(months=24)
shop_id = 2445081

2023-08-30 11:49:07.399 | DEBUG    | src.database.session:<module>:17 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


In [2]:
df = fcd.read_df(df_type=DescriptiveDF.main)
df = df[df.feature == 'creative_type']
group_column = 'feature_value'
metric_column = 'ctr'
test_func = kruskal
test_func_kwargs = {'nan_policy': 'omit'}
pd_feature_func = 'median'

/app/play/../src/utils/pd.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [2]:
main_df = fcd.read_df(df_type=DescriptiveDF.main)
# st.write(main_df[main_df.year_month.isna()])
# main_df["year_month"] = main_df.year_month.apply(lambda x: datetime.strptime(x, "%Y-%m"))


In [3]:
main_df[main_df.year_month.apply(type) != str]

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads


In [4]:
df[df.year_month.isna()]

,Unnamed: 0,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads


In [18]:
df['ctr'] = df.apply(ctr.formula_series, axis=1)

In [43]:
grouped_df = df.groupby(group_column)[metric_column]
sorted_groups = getattr(grouped_df, pd_feature_func)().sort_values(ascending=False).index.tolist()
pvalue_df = pd.DataFrame(columns=sorted_groups, index=sorted_groups)
partition = []
current_slice = [sorted_groups[0]]
for better_group, worse_group in zip(sorted_groups[:-1], sorted_groups[1:]):
    better_sample = grouped_df.get_group(better_group)
    worse_sample = grouped_df.get_group(worse_group)
    pvalue = test_func(better_sample, worse_sample, **test_func_kwargs).pvalue
    pvalue_df.loc[better_group, worse_group] = pvalue
    if pvalue < 0.1:
        partition.append(current_slice)
        current_slice = []
    current_slice.append(worse_group)
partition.append(current_slice)

In [44]:
pvalue_df

,video,carousel,dynamic,unknown,image
video,NaN,0.068048,NaN,NaN,NaN
carousel,NaN,NaN,0.623849,NaN,NaN
dynamic,NaN,NaN,NaN,0.319381,NaN
unknown,NaN,NaN,NaN,NaN,0.015999
image,NaN,NaN,NaN,NaN,NaN


In [39]:
partition = []
current_slice = [sorted_groups[0]]
for better_group, worse_group in zip(sorted_groups[:-1], sorted_groups[1:]):
    better_sample = grouped_df.get_group(better_group)
    worse_sample = grouped_df.get_group(worse_group)
    p = test_func(better_sample, worse_sample, **test_func_kwargs).pvalue
    if p < 0.1:
        partition.append(current_slice)
        current_slice = []
    current_slice.append(worse_group)
partition.append(current_slice)

In [40]:
partition

[['video'], ['carousel', 'dynamic', 'unknown'], ['image']]

In [32]:
sorted_groups

['video', 'carousel', 'dynamic', 'unknown', 'image']

In [2]:
df = fcd.get_shop_descriptive_df(db=db, shop_id=16038, start_date=fcd.start_date, end_date=fcd.end_date)

2023-08-23 12:06:28.465 | DEBUG    | src.abc.descriptive:get_shop_descriptive_df:79 - 16038: +


In [4]:
df.reset_index()

,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
0,2021-08,creative_type,carousel,39472.02,2356863.0,30647.0,417.0,81866.18,10
1,2021-08,creative_type,dynamic,7330.64,824425.0,5533.0,50.0,12367.94,1
2,2021-08,cta,False,46802.66,3181288.0,36180.0,467.0,94234.12,11
3,2021-08,discount,False,32465.40,2318380.0,26304.0,343.0,69488.28,9
4,2021-08,discount,True,14337.26,862908.0,9876.0,124.0,24745.84,2
...,...,...,...,...,...,...,...,...,...
593,2023-07,target,remarketing,9733.30,358712.0,2686.0,588.0,93628.17,9
594,2023-07,urgency,False,247202.34,15707007.0,159885.0,2384.0,427760.61,60
595,2023-07,user_addressing,False,9096.61,598525.0,4972.0,42.0,7534.20,3
596,2023-07,user_addressing,True,238105.73,15108482.0,154913.0,2342.0,420226.41,57


In [4]:
fcd.end_date

datetime.date(2023, 7, 31)

In [ ]:
df = fcd.create_and_save_main(db=db, end_date=end_date, testing=True)

2023-08-23 07:23:25.218 | DEBUG    | src.services.descriptive.descriptive_saver:create_and_save_main:72 - list_of_objects_on_s3 = []
  0%|          | 0/2 [00:00<?, ?it/s]2023-08-23 07:23:29.479 | DEBUG    | src.abc.descriptive.descriptive:get_shop_descriptive_df:26 - 16038: +
2023-08-23 07:23:29.481 | ERROR    | src.services.descriptive.descriptive_saver:create_and_save_main:132 - Error for shop id 16038: 
 local variable 'df' referenced before assignment
 50%|█████     | 1/2 [00:04<00:04,  4.25s/it]2023-08-23 07:23:38.422 | DEBUG    | src.abc.descriptive.descriptive:get_shop_descriptive_df:26 - 44301396: +
2023-08-23 07:23:38.424 | ERROR    | src.services.descriptive.descriptive_saver:create_and_save_main:132 - Error for shop id 44301396: 
 local variable 'df' referenced before assignment
100%|██████████| 2/2 [00:13<00:00,  6.60s/it]


In [2]:
df = fcd.get_shop_descriptive_df(db=db, shop_id=16038, start_date=start_date, end_date=end_date)

2023-08-23 07:27:32.571 | DEBUG    | src.abc.descriptive.descriptive:get_shop_descriptive_df:26 - 16038: +


In [5]:
df[df.index.get_level_values('feature').isin([fcd.fake_feature])]

,,,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
year_month,feature,feature_value,,,,,,
2021-08,fake_feature,.,46802.66,3181288.0,36180.0,467.0,94234.12,11
2021-09,fake_feature,.,14685.42,644275.0,8119.0,182.0,33582.36,10
2021-10,fake_feature,.,14405.83,657867.0,8443.0,165.0,32914.08,14
2021-11,fake_feature,.,29712.06,1085548.0,12703.0,283.0,65533.72,16
2021-12,fake_feature,.,37576.15,2064083.0,25338.0,230.0,55109.50,28
2022-01,fake_feature,.,128957.96,7752832.0,88656.0,915.0,207634.21,28
2022-02,fake_feature,.,190522.86,8395216.0,85671.0,1228.0,265496.12,19
2022-03,fake_feature,.,239163.02,9819873.0,97975.0,1717.0,372800.38,24
2022-04,fake_feature,.,82381.74,2804193.0,34621.0,858.0,155961.30,18


In [2]:
main_df = fcd.read_df('main', end_date=end_date)

In [3]:
main_df.apply(cr.formula_series, axis=1)

0         7.784149
1         0.000000
2         0.000000
3         0.000000
4         8.314337
            ...   
298913    0.609794
298914    0.384568
298915    0.664939
298916    0.556147
298917    0.475170
Length: 298918, dtype: float64

In [4]:
df = pd.DataFrame()

In [5]:
df[cr] = [5]

In [7]:
df.loc[0, cr]

5

In [8]:
cr

In [17]:
main_df.purch_value_USD.quantile(q=0.35)

64.55234353073253

In [18]:
conversion_df = main_df[main_df.purch_value_USD > 0]

In [20]:
conversion_df.clicks.describe()

count    2.048710e+05
mean     2.017217e+04
std      1.108909e+05
min      0.000000e+00
25%      7.220000e+02
50%      3.058000e+03
75%      1.141450e+04
max      4.408504e+06
Name: clicks, dtype: float64

In [101]:
normalised_relative_sum_by_value = relative_sum_by_value.div(n_shops_by_month_and_feature, axis=0)

In [104]:
absolute_sum_by_value = absolute_sum_by_shop_and_value.reset_index().groupby(['year_month', 'feature', 'feature_value'])[fcd.metric_columns].sum()

In [106]:
normalised_relative_sum_by_value.reset_index().groupby(['year_month', 'feature'])[fcd.metric_columns].sum()
normalised_relative_sum_by_value.rename(columns = {col: col+'_by_shop' for col in fcd.metric_columns}, inplace=True)

In [107]:
summary_df = absolute_sum_by_value.join(normalised_relative_sum_by_value)

In [108]:
summary_df

spend_USD         impr  \
year_month feature         feature_value                              
2021-08    creative_type   carousel       1.385065e+06  189680897.0   
                           dynamic        2.015461e+06  231883453.0   
                           image          1.127209e+06  179940512.0   
                           unknown        1.303205e+06  193829477.0   
                           video          1.305978e+06   95865870.0   
...                                                ...          ...   
2023-07    urgency         True           6.186890e+05  110600249.0   
           user_addressing False          1.933799e+06  372190025.0   
                           True           3.717627e+06  418143650.0   
           weasel_words    False          4.885422e+06  715353641.0   
                           True           7.660042e+05   74980034.0   

                                             clicks     purch  \
year_month feature         feature_value                        
2021-08    creative_type   carousel       2731361.0   36152.0   
                           dynamic        2175074.0   41627.0   
                           image          1294595.0   22434.0   
                           unknown        2306950.0   40886.0   
                           video          1101071.0   31390.0   
...                                             ...       ...   
2023-07    urgency         True           1707297.0   30406.0   
           user_addressing False          4291428.0   67180.0   
                           True           6660496.0  112670.0   
           weasel_words    False          9950143.0  161739.0   
                           True           1001781.0   18111.0   

                                          purch_value_USD  n_ads  \
year_month feature         feature_value                           
2021-08    creative_type   carousel          2.931933e+06   4930   
                           dynamic           2.838939e+06   9762   
                           image             1.328029e+06   7247   
                           unknown           2.091133e+06   8597   
                           video             1.556334e+06   4154   
...                                                   ...    ...   
2023-07    urgency         True              1.747118e+06   2382   
           user_addressing False             5.651549e+06   6218   
                           True              9.267915e+06  10103   
           weasel_words    False             1.369261e+07  13971   
                           True              1.226858e+06   2350   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel                0.262008      0.249020   
                           dynamic                 0.234237      0.241022   
                           image                   0.173454      0.175171   
                           unknown                 0.227982      0.235680   
                           video                   0.102319      0.099107   
...                                                     ...           ...   
2023-07    urgency         True                    0.123016      0.117647   
           user_addressing False                   0.517387      0.526622   
                           True                    0.482613      0.473378   
           weasel_words    False                   0.901555      0.898092   
                           True                    0.098445      0.101908   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel             0.256854       0.331802   
                           dynamic              0.237936       0.264744   
                           image                0.177017       0.145836   
             

In [81]:
n_shops_by_month_and_feature = relative_sum_by_value.reset_index().groupby(['year_month', 'feature'])['n_ads'].sum()

In [86]:
relative_sum

spend_USD      impr  \
shop_id   year_month feature         feature_value                        
2         2021-08    creative_type   carousel        0.238184  0.274308   
                                     dynamic         0.636367  0.529109   
                                     image           0.004610  0.003239   
                                     video           0.120839  0.193345   
                     cta             False           1.000000  1.000000   
...                                                       ...       ...   
135821055 2023-07    target          acquisition     1.000000  1.000000   
                     urgency         False           1.000000  1.000000   
                     user_addressing False           0.963964  0.980708   
                                     True            0.036036  0.019292   
                     weasel_words    False           1.000000  1.000000   

                                                      clicks     purch  \
shop_id   year_month feature         feature_value                       
2         2021-08    creative_type   carousel       0.314626  0.329970   
                                     dynamic        0.590841  0.590817   
                                     image          0.003731  0.006054   
                                     video          0.090802  0.073158   
                     cta             False          1.000000  1.000000   
...                                                      ...       ...   
135821055 2023-07    target          acquisition    1.000000  1.000000   
                     urgency         False          1.000000  1.000000   
                     user_addressing False          0.976217  0.963731   
                                     True           0.023783  0.036269   
                     weasel_words    False          1.000000  1.000000   

                                                    purch_value_USD     n_ads  
shop_id   year_month feature         feature_value                             
2         2021-08    creative_type   carousel              0.317657  0.182692  
                                     dynamic               0.602077  0.706731  
                                     image                 0.014612  0.028846  
                                     video                 0.065654  0.081731  
                     cta             False                 1.000000  1.000000  
...                                                             ...       ...  
135821055 2023-07    target          acquisition           1.000000  1.000000  
                     urgency         False                 1.000000  1.000000  
                     user_addressing False                 0.957599  0.916667  
                                     True                  0.042401  0.083333  
                     weasel_words    False                 1.000000  1.000000  

[298918 rows x 6 columns]

In [85]:
relative_sum / n_shops_by_month_and_feature

ValueError: cannot join with no overlapping index names

In [70]:
relative_sum

spend_USD        impr      clicks  \
year_month feature         feature_value                                       
2021-08    creative_type   carousel       137.816231  131.233651  133.306997   
                           dynamic        123.208917  127.018405  123.488860   
                           image           91.236672   92.314925   91.871912   
                           unknown        119.918445  124.203611  117.642960   
                           video           53.819735   52.229409   52.689270   
...                                              ...         ...         ...   
2023-07    urgency         True            53.511801   51.176310   50.696262   
           user_addressing False          225.063157  229.080567  225.872331   
                           True           209.936843  205.919433  205.127669   
           weasel_words    False          392.176559  390.670208  388.550778   
                           True            42.823441   44.329792   42.449222   

                                               purch  purch_value_USD  \
year_month feature         feature_value                                
2021-08    creative_type   carousel       130.398327       127.178601   
                           dynamic        104.044208       101.483269   
                           image           57.313580        53.986773   
                           unknown         59.682194        59.036732   
                           video           41.561691        42.314625   
...                                              ...              ...   
2023-07    urgency         True            49.721755        50.098164   
           user_addressing False          160.606120       160.695168   
                           True           160.393880       159.304832   
           weasel_words    False          290.348869       288.723133   
                           True            30.651131        31.276867   

                                               n_ads  
year_month feature         feature_value              
2021-08    creative_type   carousel       120.216309  
                           dynamic        124.673185  
                           image           98.655759  
                           unknown        128.104435  
                           video           57.350311  
...                                              ...  
2023-07    urgency         True            50.299489  
           user_addressing False          220.707686  
                           True           215.292314  
           weasel_words    False          394.168564  
                           True            41.831436  

[816 rows x 6 columns]

In [2]:
fcd.create_and_save_main(db=db, end_date=date.today(), force_from_scratch=True)

100%|██████████| 2/2 [00:16<00:00,  8.07s/it]


In [4]:
main_df = fcd.read_df('main', end_date=end_date)


,Unnamed: 0,year_month,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,feature,shop_id
41,41,2023-06,False,136710.680000,9415026.0,83186.0,1112.0,181100.080000,20,starts_with_question,16038
42,42,2023-06,True,495.400000,17727.0,324.0,63.0,8721.330000,1,starts_with_question,16038
67,22,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,cta,16038
97,28,2023-06,True,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,emoji,16038
124,25,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,weasel_words,16038
150,24,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,urgency,16038
184,32,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,free_shipping,16038
215,29,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,fact_words,16038
261,43,2023-06,False,70862.480000,5757317.0,51503.0,501.0,77518.980000,9,discount,16038
262,44,2023-06,True,66343.600000,3675436.0,32007.0,674.0,112302.430000,12,discount,16038


In [2]:
fcd.create_and_save_summary(end_date=end_date)

2023-08-21 12:49:52.905 | DEBUG    | src.services.descriptive.descriptive_saver:create_and_save_summary:36 - ['year_month', 'feature', 'feature_value']


In [2]:
summary_df = fcd.read_df('summary', end_date=end_date)
summary_df

,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop
0,2021-08,creative_type,carousel,42030.279927,2720853.0,33467.0,502.0,90318.793686,45,2,0.794254,0.678537,0.782195,0.738235,0.796761,0.398230
1,2021-08,creative_type,dynamic,10887.664708,1289029.0,9319.0,178.0,23038.652777,68,2,0.205746,0.321463,0.217805,0.261765,0.203239,0.601770
2,2021-08,cta,False,52917.944635,4009882.0,42786.0,680.0,113357.446463,113,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,2021-08,discount,False,35690.936214,2774374.0,29747.0,455.0,78983.838743,39,2,0.674458,0.691884,0.695251,0.669118,0.696768,0.345133
4,2021-08,discount,True,17227.008421,1235508.0,13039.0,225.0,34373.607720,74,2,0.325542,0.308116,0.304749,0.330882,0.303232,0.654867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,2023-07,urgency,False,259475.009953,17869595.0,175611.0,3166.0,477205.144986,69,2,0.971241,0.926095,0.955399,0.881894,0.939683,0.945205
712,2023-07,urgency,True,7683.105525,1426041.0,8198.0,424.0,30631.421755,4,1,0.028759,0.073905,0.044601,0.118106,0.060317,0.054795
713,2023-07,user_addressing,False,9096.610000,598525.0,4972.0,42.0,7534.200000,3,1,0.034050,0.031019,0.027050,0.011699,0.014836,0.041096
714,2023-07,user_addressing,True,258061.505478,18697111.0,178837.0,3548.0,500302.366742,70,2,0.965950,0.968981,0.972950,0.988301,0.985164,0.958904


In [5]:
summary_df[summary_df.feature == 'creative_type'].groupby(['year_month', 'feature']).sum()

,,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop
year_month,feature,,,,,,,,,,,,,,
2021-08,creative_type,carouseldynamic,52917.944635,4009882.0,42786.0,680.0,113357.446463,113,4,1.0,1.0,1.0,1.0,1.0,1.0
2021-09,creative_type,carouseldynamic,24165.890637,1971449.0,17117.0,630.0,63470.895325,66,3,1.0,1.0,1.0,1.0,1.0,1.0
2021-10,creative_type,carouseldynamic,26427.459235,2223481.0,20100.0,824.0,87163.549962,82,3,1.0,1.0,1.0,1.0,1.0,1.0
2021-11,creative_type,carouseldynamic,52778.190299,3587435.0,31512.0,2046.0,237975.227778,98,4,1.0,1.0,1.0,1.0,1.0,1.0
2021-12,creative_type,carouseldynamicimageunknown,46631.926584,3184183.0,35885.0,876.0,105553.249608,178,6,1.0,1.0,1.0,1.0,1.0,1.0
2022-01,creative_type,carouseldynamicimageunknownvideo,137052.389716,9453957.0,102873.0,1268.0,239731.618990,121,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-02,creative_type,carouseldynamicimagevideo,195549.138820,9303403.0,90777.0,1487.0,288846.630864,65,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-03,creative_type,carouseldynamicimagevideo,277006.652355,14503907.0,134153.0,4358.0,598468.803013,312,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-04,creative_type,carouseldynamicimagevideo,89604.532922,3828222.0,42996.0,1005.0,168579.315022,165,7,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
main_df = fcd.get_main_df(end_date=end_date)
main_df

spend_USD         impr  \
year_month feature         feature_value                              
2021-08    creative_type   carousel       1.385117e+06  189679520.0   
                           dynamic        2.015207e+06  231822297.0   
                           image          1.122925e+06  179716465.0   
                           unknown        1.297635e+06  193497346.0   
                           video          1.305978e+06   95865870.0   
...                                                ...          ...   
2023-07    urgency         True           6.187278e+05  110600249.0   
           user_addressing False          1.931415e+06  371170936.0   
                           True           3.717662e+06  418116014.0   
           weasel_words    False          4.883056e+06  714306181.0   
                           True           7.660223e+05   74980769.0   

                                             clicks     purch  \
year_month feature         feature_value                        
2021-08    creative_type   carousel       2731351.0   36151.0   
                           dynamic        2174472.0   41599.0   
                           image          1288717.0   22366.0   
                           unknown        2302229.0   40817.0   
                           video          1101071.0   31390.0   
...                                             ...       ...   
2023-07    urgency         True           1707297.0   30406.0   
           user_addressing False          4285518.0   67128.0   
                           True           6660266.0  112665.0   
           weasel_words    False          9943996.0  161682.0   
                           True           1001788.0   18111.0   

                                          purch_value_USD    n_ads  \
year_month feature         feature_value                             
2021-08    creative_type   carousel          2.932261e+06   4929.0   
                           dynamic           2.836585e+06   9757.0   
                           image             1.323270e+06   7237.0   
                           unknown           2.087038e+06   8565.0   
                           video             1.556321e+06   4154.0   
...                                                   ...      ...   
2023-07    urgency         True              1.747109e+06   2382.0   
           user_addressing False             5.646147e+06   6168.0   
                           True              9.267490e+06  10101.0   
           weasel_words    False             1.368683e+07  13919.0   
                           True              1.226810e+06   2350.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel              137.795773    131.217613   
                           dynamic               122.230413    126.040613   
                           image                  90.810532     91.908048   
                           unknown               118.337687    122.586704   
                           video                  53.825595     52.247022   
...                                                     ...           ...   
2023-07    urgency         True                   53.511801     51.176310   
           user_addressing False                 220.288345    224.220472   
                           True                  209.711655    205.779528   
           weasel_words    False                 387.141526    385.659054   
                           True                   42.858474     44.340946   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel           133.292906     130.363844   
                           dynamic            122.505200     103.078691   
                           image               91.310972      

In [5]:
set(main_df.index.get_level_values(level='feature').tolist())

{'creative_type',
 'cta',
 'discount',
 'emoji',
 'fact_words',
 'fake_column_for_total_sum',
 'free_shipping',
 'hashtag',
 'link',
 'percentage',
 'price',
 'starts_with_question',
 'target',
 'urgency',
 'user_addressing',
 'weasel_words'}

In [6]:
main_df.reset_index(inplace=True)

In [8]:
df_fw = fcd.get_shop_descriptive_df(db=db, shop_id=16038, end_date=end_date, start_date = start_date)
df_dr = fcd.get_shop_descriptive_df(db=db, shop_id=44301396, end_date=end_date, start_date = start_date)
df_test = df_fw.add(df_dr, fill_value=0)
df_test

2023-08-17 11:13:28.909 | DEBUG    | src.services.descriptive.descriptive_saver:get_shop_descriptive_df:115 - 16038: +
2023-08-17 11:13:34.219 | DEBUG    | src.services.descriptive.descriptive_saver:get_shop_descriptive_df:115 - 44301396: +


spend_USD        impr    clicks  \
year_month feature         feature_value                                        
2021-08    creative_type   carousel        42026.960098   2720853.0   33467.0   
                           dynamic         10883.048792   1289029.0    9319.0   
           cta             False           52910.008889   4009882.0   42786.0   
           discount        False           35686.750467   2774374.0   29747.0   
                           True            17223.258422   1235508.0   13039.0   
...                                                 ...         ...       ...   
2023-07    urgency         False          259459.083828  17869595.0  175611.0   
                           True             7673.135233   1426041.0    8198.0   
           user_addressing False            9096.610000    598525.0    4972.0   
                           True           258035.609061  18697111.0  178837.0   
           weasel_words    False          267132.219061  19295636.0  183809.0   

                                           purch  purch_value_USD  n_ads  \
year_month feature         feature_value                                   
2021-08    creative_type   carousel        502.0     90307.824811   45.0   
                           dynamic         178.0     23024.805496   68.0   
           cta             False           680.0    113332.630308  113.0   
           discount        False           455.0     78971.516448   39.0   
                           True            225.0     34361.113859   74.0   
...                                          ...              ...    ...   
2023-07    urgency         False          3166.0    477140.981293   69.0   
                           True            424.0     30591.671656    4.0   
           user_addressing False            42.0      7534.200000    3.0   
                           True           3548.0    500198.452949   70.0   
           weasel_words    False          3590.0    507732.652949   73.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel                1.261710      1.180138   
                           dynamic                 0.738290      0.819862   
           cta             False                   2.000000      2.000000   
           discount        False                   1.221120      1.279078   
                           True                    0.778880      0.720922   
...                                                     ...           ...   
2023-07    urgency         False                   1.614993      1.602622   
                           True                    0.385007      0.397378   
           user_addressing False                   0.036798      0.038106   
                           True                    1.963202      1.961894   
           weasel_words    False                   2.000000      2.000000   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel             1.273955       1.291995   
                           dynamic              0.726045       0.708005   
           cta             False                2.000000       2.000000   
           discount        False                1.248224       1.260297   
                           True                 0.751776       0.739703   
...                                                  ...            ...   
2023-07    urgency         False                1.657332       1.648425   
                           True                 0.342668       0.351575   
           user_addressing False                0.031097       0.017617   
                           True                 1.968903       1.982383   
           weasel_words    False                2.000000       2.000000   

                         

In [9]:
main_df.subtract(df_test)

spend_USD  impr  clicks  purch  \
year_month feature         feature_value                                   
2021-08    creative_type   carousel             0.0   0.0     0.0    0.0   
                           dynamic              0.0   0.0     0.0    0.0   
           cta             False                0.0   0.0     0.0    0.0   
           discount        False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
...                                             ...   ...     ...    ...   
2023-07    urgency         False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
           user_addressing False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
           weasel_words    False                0.0   0.0     0.0    0.0   

                                          purch_value_USD  n_ads  \
year_month feature         feature_value                           
2021-08    creative_type   carousel          0.000000e+00    0.0   
                           dynamic           0.000000e+00    0.0   
           cta             False             0.000000e+00    0.0   
           discount        False             0.000000e+00    0.0   
                           True              0.000000e+00    0.0   
...                                                   ...    ...   
2023-07    urgency         False             0.000000e+00    0.0   
                           True              0.000000e+00    0.0   
           user_addressing False             0.000000e+00    0.0   
                           True              5.820766e-11    0.0   
           weasel_words    False             0.000000e+00    0.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel            0.000000e+00  0.000000e+00   
                           dynamic             0.000000e+00  0.000000e+00   
           cta             False               0.000000e+00  0.000000e+00   
           discount        False               0.000000e+00  2.220446e-16   
                           True                0.000000e+00  0.000000e+00   
...                                                     ...           ...   
2023-07    urgency         False               0.000000e+00  0.000000e+00   
                           True                0.000000e+00  0.000000e+00   
           user_addressing False              -2.081668e-17 -1.387779e-17   
                           True                0.000000e+00  0.000000e+00   
           weasel_words    False               0.000000e+00  0.000000e+00   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel        -2.220446e-16   0.000000e+00   
                           dynamic          0.000000e+00   0.000000e+00   
           cta             False            0.000000e+00   0.000000e+00   
           discount        False            0.000000e+00   0.000000e+00   
                           True             0.000000e+00   0.000000e+00   
...                                                  ...            ...   
2023-07    urgency         False            0.000000e+00   0.000000e+00   
                           True            -5.551115e-17   0.000000e+00   
           user_addressing False           -3.122502e-17  -2.775558e-17   
                           True            -2.220446e-16   0.000000e+00   
           weasel_words    False            0.000000e+00   0.000000e+00   

                                          purch_value_USD_by_shop  \
year_month feature         feature_value                            
2021-08    creative_type   carousel                 